In [1]:
from src.utils import remove_background
from src.utils import INPUT_PATH, OUTPUT_PATH, remove_background, create_octree_image, jaccard_similarity_coefficient, create_palette_image
from os import listdir
from os.path import isfile, join

In [2]:
REMOVE_BACKGROUND = True

## Remove background from images

In [3]:
# Remove backgroud from image
if (REMOVE_BACKGROUND):
    files = [f for f in listdir(INPUT_PATH) if isfile(join(INPUT_PATH, f))]

    for idx,img in enumerate(files):
        remove_background(img)


100%|########################################| 176M/176M [00:00<00:00, 176GB/s]


## Load image to octree

In [5]:
octree, img_info = create_octree_image('./dataset/no_bg/2.png', 8)

# 256 colors for 8 bits per pixel output image
palette = octree.make_palette(256)

# create palette for 256 color max and save to file
create_palette_image(palette, './test_palette.png', 16, 16)


# save output image
out_image = Image.new('RGB', (width, height))
out_pixels = out_image.load()

for j in range(height):
    for i in range(width):
        index = octree.get_palette_index(Color(*pixels[i, j]))
        color = palette[index]
        out_pixels[i, j] = (color.red, color.green, color.blue)
out_image.save('./test_out.png')

In [13]:
oct1 = create_octree_image('./dataset/no_bg/1.png', 6)
oct2 = create_octree_image('./dataset/no_bg/2.png', 6)
oct3 = create_octree_image('./dataset/no_bg/3.png', 6)


# 256 colors for 8 bits per pixel output image
palette1 = oct1.make_optimized_palette(512)
palette2 = oct2.make_optimized_palette(512)
palette3 = oct3.make_optimized_palette(512)

set1 = oct1.node_id_set()
set2 = oct2.node_id_set()
set3 = oct3.node_id_set()

print("Jaccard similarity coefficient between 1 and 2: ", jaccard_similarity_coefficient(set1,set2))
print("Jaccard similarity coefficient between 1 and 3: ", jaccard_similarity_coefficient(set1,set3))
print("Jaccard similarity coefficient between 2 and 3: ", jaccard_similarity_coefficient(set2,set3))

Jaccard similarity coefficient between 1 and 2:  0.26413255360623783
Jaccard similarity coefficient between 1 and 3:  0.3231083844580777
Jaccard similarity coefficient between 2 and 3:  0.2997061704211557
